In [1]:
import socket
import mysql.connector
myconn=mysql.connector.connect(host="localhost",user="root",passwd="aman37246",database="project")
cur=myconn.cursor()

def ServerIn():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    host = socket.gethostname()
    port = 12345
    s.bind((host, port))
    s.listen(5)
    print('Server Listening...')
    print("Waiting for Connection")

    conn, addr = s.accept()
    print("Got connection from: ", addr)
    g=1
    while (g):
        choice = conn.recv(1024).decode()
        if (choice == "1"):
            username = conn.recv(1024).decode()
            password = conn.recv(1024).decode()
            repassword=conn.recv(1024).decode()
            email = conn.recv(1024).decode()
            gender = conn.recv(1024).decode()
            college = conn.recv(1024).decode()
            create = accountcreation(username,password,repassword,email,gender,college)
            conn.sendall(create.encode())
        elif choice == "2":
            username = conn.recv(1024).decode()
            password = conn.recv(1024).decode()
            login = Login(username, password)
            conn.sendall(login.encode())
            if(login=="1"):
                k=1
                while k:
                    ch=conn.recv(1024).decode()
                    if ch == "3":
                        n_email = conn.recv(1024).decode()
                        n_gender = conn.recv(1024).decode()
                        n_college = conn.recv(1024).decode()
                        upd = update(username,n_email,n_gender,n_college)
                        conn.sendall(upd.encode())
                    elif ch == "4":
                        v = view(username,password)
                        conn.sendall(v.encode())
                    elif ch == "5":
                        k=0
                        d = delete(username)
                        conn.sendall(d.encode())
                    elif ch == "6":
                        reply=conn.recv(1024).decode()
                        Logout = logout(reply)
                        conn.sendall(Logout.encode())
                        if(Logout=="-1"):
                            k=0
                        
        elif choice == "0":
            g=1
            conn.close()
            print("Connection closed!")
            break;
        else:
            print("Invalid Choice")

def accountcreation(username,password,repassword,email,gender,college):
    list=[]
    cur.execute("select username,password from finaldata")
    for i in cur:
        list.append(i)
    if(username,password) in list:
        return "\nUsername already exists!!"
    else:
        if(password==repassword):
            sql="insert into finaldata(username,password,repassword,email,gender,college) values (%s,%s,%s,%s,%s,%s)"
            val=(username,password,repassword,email,gender,college)
            cur.execute(sql,val)
            myconn.commit()
            return "\nAccount created successfully"
        else:
            return "\nPassword doesn't match"
    
def Login(username,password):
    list=[]
    cur.execute("select username,password from finaldata")
    for i in cur:
        list.append(i)
    if(username,password) in list:
        return "1"
    else:
        return "2"

def update(username,n_email,n_gender,n_college): 
    try:
        sql="Update finaldata SET email=%s,gender=%s,college=%s where username=%s"
        val=(n_email,n_gender,n_college,username)
        cur.execute(sql,val)
        myconn.commit()
        return "\nAccount details updated successfully"
    except:
        myconn.rollback()

def view(username,password):
    cur.execute("select * from finaldata where username=%s and password=%s",(username,password))
    li=[]
    for i in cur:
        li=i
    myconn.commit()     
    st=("username="+li[0]+" "+"email="+li[3]+" "+"gender="+li[4]+" "+"college="+li[5])
    return st
    
def delete(username):
    try:
        sql="DELETE FROM finaldata WHERE username=%s"
        val=username
        cur.execute(sql,(val,))
        myconn.commit()
        return "\nAccount deleted successfully"
    except:
        myconn.rollback()

def logout(reply):
    if reply == "y":
        return "-1"
    elif reply == "n":
        return "-2"
    else:
        return "-3"

ServerIn()

Server Listening...
Waiting for Connection
Got connection from:  ('192.168.43.157', 59710)


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine